Frequentist Approach relies on classification models
such Logistics Regression, Decision Trees, Random Forest, XG Boost to find the exact coefficietns value for customer classification

For Customer churn, the output should be binary i.e. whether customer churned or not. 

Since the data is related to customer subscription, customer needs to explicitly sign out and thus we have clear signal that the customer has churned in contarast to ecommerce where depending upon customer behaviour, we get a tendancy that customer might have churned and thus bayesian churn models are more applicable in such scenarios. 

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import os

#Add Data

In [ ]:

inputData =  pd.read_csv('C:/Users/sahil/Desktop/Case/advance.csv')
#training_data = inputData.sample(frac=0.8, random_state=25)
#testing_data = inputData.drop(training_data.index)
#print(inputData.columns)
#del inputData['Not Cancelled 2016 Subscription']
#del inputData['ACCT_ID']
#del inputData['USERS_2016']
#del inputData['Total Revenue 2015']

In [ ]:

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
inputData['En_FIRST_YR_CUSTOMER'] = label_encoder.fit_transform(inputData['FIRST_YR_CUSTOMER'] )
inputData['En_COUNTRY'] = label_encoder.fit_transform(inputData['COUNTRY'] )
inputData['En_ACCT_MGR_STATUS'] = label_encoder.fit_transform(inputData['ACCT_MGR_STATUS'] )
inputData['En_ACCT_REVENUE_CAT'] = label_encoder.fit_transform(inputData['ACCT_REVENUE_CAT'] )
inputData['En_ACCT_EMPLOYEE_CAT'] = label_encoder.fit_transform(inputData['ACCT_EMPLOYEE_CAT'] )
inputData['En_ACCT_TYPE'] = label_encoder.fit_transform(inputData['ACCT_TYPE'] )

inputData.columns

#Normalise the data since it is a model for logistics regressions

In [ ]:

cols = ['MEDICAL_REVENUE_2015',
       'SCIENTIFIC_REVENUE_2015', 'TECHNICAL_REVENUE_2015',
       'MEDICAL_NO_SESSIONS_2015', 'SCIENTIFIC_NO_SESSIONS_2015',
       'TECHNICAL_NO_SESSIONS_2015', 'USERS_2015', 'En_FIRST_YR_CUSTOMER', 'En_COUNTRY',
       'En_ACCT_MGR_STATUS', 'En_ACCT_REVENUE_CAT', 'En_ACCT_EMPLOYEE_CAT',
       'En_ACCT_TYPE']
X = inputData[cols]
y = inputData.loc[:, inputData.columns == 'Cancelled 2016 subscription']

from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler()
X_norm = norm.fit_transform(X)
X_norm = pd.DataFrame(X_norm, columns = X.columns)
X_norm.head()

X_norm.describe()


#apply SelectKBest class to extract top 10 best features

In [ ]:

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=5)
fit = bestfeatures.fit(X_norm,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_norm.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features
featureScores[featureScores.Score > featureScores.Score.mean()]




In [ ]:

cols = featureScores[featureScores.Score > featureScores.Score.mean()].Specs
X = X_norm[cols]

We need to Balance the data using SMOT so that there is no bias for Churned out customers

In [ ]:
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

#Logistics Regressions

In [ ]:

from sklearn.linear_model import LogisticRegression
lr_std = LogisticRegression()
info = lr_std.fit(X_train, y_train)
y_pred = lr_std.predict(X_test)
print('Accuracy of logistic regression on test set with standardized features: {:.2f}'.format(lr_std.score(X_test, y_test)))


#Output

In [ ]:

import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit(maxiter=2000)
print(result.summary())


#Random Forest

In [ ]:

from sklearn.model_selection import train_test_split
X = X_norm
#implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)


In [ ]:

from sklearn.ensemble import RandomForestClassifier
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)
f_i = list(zip(X.columns,rfc.feature_importances_))
f_i.sort(key = lambda x : x[1])
plt.barh([x[0] for x in f_i],[x[1] for x in f_i])

plt.show()

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring="roc_auc")



In [ ]:

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())


In [ ]:


rfc = RandomForestClassifier(n_estimators=200, max_depth=140, max_features='sqrt')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())


#Adaboost Classifier

In [ ]:

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
ada_classifier=AdaBoostClassifier()
ada_classifier.fit(X_train, y_train)
ytest_pred = ada_classifier.predict_proba(X_test)
print('Adaboost test roc-auc: {}'.format(roc_auc_score(y_test, ytest_pred[:,1])))



#Decision Trees

In [ ]:

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

X = inputData.loc[:, inputData.columns != 'Cancelled 2016 subscription']

X = X[['MEDICAL_REVENUE_2015', 'TECHNICAL_REVENUE_2015',
       'MEDICAL_NO_SESSIONS_2015', 'USERS_2015', 'FIRST_YR_CUSTOMER']]

y = inputData.loc[:, inputData.columns == 'Cancelled 2016 subscription']

regr = DecisionTreeRegressor(max_depth=4, random_state=1234)
model = regr.fit(X, y)

#Output

In [ ]:

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn import metrics

X = inputData.loc[:, inputData.columns != 'Cancelled 2016 subscription']

X = X[['MEDICAL_REVENUE_2015', 'TECHNICAL_REVENUE_2015',
       'MEDICAL_NO_SESSIONS_2015', 'USERS_2015', 'FIRST_YR_CUSTOMER']]

y = inputData.loc[:, inputData.columns == 'Cancelled 2016 subscription']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

regr = DecisionTreeRegressor(max_depth=4, random_state=1234)
model = regr.fit(X_train, y_train)

Decision_tree_predict = model.predict(X_test)>0.5
Decision_tree_predict
#print(classification_report(y_test, Decision_tree_predict))
metrics.accuracy_score(y_test, Decision_tree_predict)


In [ ]:

fig = plt.figure(figsize=(100,100))
_ = tree.plot_tree(regr, feature_names= ['MEDICAL_REVENUE_2015', 'TECHNICAL_REVENUE_2015',
       'MEDICAL_NO_SESSIONS_2015', 'USERS_2015', 'En_FIRST_YR_CUSTOMER']
                   , filled=True)

